## En construcción (no puedo hacer andar los Nedelec) ##

# Capacity Project

Writen by: Oscar Reula (oreula@gmail.com)

This code is writen in Fenics/Python, it is used to compute the resonant frequencies and eigenvectors of a cavity. At the moment is 2-dimensional, that is it corresponds to infinite long structures in the  ẑ direction. 
This restriction is mainly due to have a code that can run in home computers in very short time.

First the theory: we need to solve the following eigenfuntion problem for the electric field:

$\nabla \wedge (\nabla \wedge \vec{E})  = \lambda \vec{E} \quad\quad\text{ in } \Omega$

$\vec{E} \wedge \hat{n}= 0 \text{ on } \partial\Omega$

Where $\Omega$ is some 2-dimensional region, we think for this problem as the electric field having zero $\hat{z}$ component. Here $\lambda = \frac{\omega^2}{c^2}$ with $\omega$ the mode frequency and $c$ the speed of light.
Notice that if the take the divergence of the right hand side of the equation we get identically zero, thus, all the modes must have zero divergence, but also implies that the operator at the right hand side has a big kernel.
Thus, to solve the problem as a variational one we must restrict the space to that of vectors with zero divergence.
It is only in this space that the operator is elliptic.

In order to use finite elements we need to express the problem as a weak one:

$$
\int_{\Omega} \nabla \wedge \vec{E}\,\cdot \nabla \wedge \vec{V}\, {\rm d} x
= \lambda \int_{\Omega} \vec{E} \cdot \vec{V}\, {\rm d} x \quad \forall \ \vec{V}\in H_0(\nabla \wedge),
$$

Where the space $H_0(\nabla \wedge)$ is the space of vectors which have zero divergence. 

We shall have a test case, namely:

\begin{align}
    \nabla \wedge (\nabla \wedge \vec{E})  = \lambda \vec{E} \quad\quad\text{ in } \Omega \quad\quad\quad
    \Omega = (0,\pi)\times(0,\pi)
\end{align}

For this case we know the solutions: (see for instance: http://www.famaf.unc.edu.ar/~reula/Docencia/Electromagnetismo/electrodynamics.pdf, Sec. 19.1)

$\vec{E}(x,y) = (m\cos(n x)\sin(m y), -n\cos(m y)\sin(n x)), \quad \lambda = m^2 + n^2,$ $\quad \lambda = 1, 1, 2, 4, 4, 5, 5, 8, 9, 9, 10, 10, 13, 13, \dots$

We use this case to check that what we are doing is correct. 

We are going to compute the eigenvectors for regions with several disconected boundaries. In that case there are an infinite set of zero mode solutions. The equations for the zero modes are just Laplace's equation for a potential,
where the boundary condition is that in each boundary the potential is constant. That problem as a n-parameter family of solutions, where the parameters are the potential's value at the n boundaries. Taking out a normalization constant we see that there remain n-1 parameters in the continuum. If the potentials are not all equal we then have a non-zero electric field and so a zero-mode for each set of parameter values.

In [1]:
import Pkg; Pkg.activate("../Tarea_5/gridap_makie")
using Gridap
using Gridap.FESpaces
using GridapMakie, GLMakie
#using CairoMakie
import Pkg; Pkg.add("WGLMakie")
using WGLMakie
using FileIO
#using Plots
#mkdir("models")
#mkdir("images")
using GridapGmsh
using Gmsh
using LinearAlgebra
using SparseArrays
using SuiteSparse

#import Pkg; Pkg.add("Arpack")
using Arpack

  Activating project at `~/Docencia/PDE/pde/2022/Numéricos/Tarea_5/gridap_makie`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Docencia/PDE/pde/2022/Numéricos/Tarea_5/gridap_makie/Project.toml`
  No Changes to `~/Docencia/PDE/pde/2022/Numéricos/Tarea_5/gridap_makie/Manifest.toml`


In [23]:
D=2
n = 20
domain = (0,π,0,π)
partition = (n,n)
model = CartesianDiscreteModel(domain, partition)

labels = get_face_labeling(model)
add_tag_from_tags!(labels,"diri_all",[1,2,3,4,5,6,7,8]);
dirichlet_tags=["diri_all"]
#dirichlet_values = [0.0,0.0]
dirichlet_values = VectorValue(0.0,0.0)

VectorValue{2, Float64}(0.0, 0.0)

In [24]:
Ω = Triangulation(model)
degree = 2
dΩ = Measure(Ω,degree)

Measure()

In [25]:
fig, ax = plot(Ω)
ax.aspect = AxisAspect(1) # this makes the region with fixed aspect, so it might not give the correct ratios
wireframe!(Ω, color=:black, linewidth=1)
scatter!(Ω, marker=:star8, markersize=4, color=:blue)
fig

In [26]:
D = 2
order = 2
reffeₑ = ReferenceFE(lagrangian, VectorValue{2,Float64}, order) #THIS WORKS
#reffeₑ = ReferenceFE(lagrangian, VectorValue{2,Complex64}, order) #THIS DOES NOT WORKS
#reffeₑ = ReferenceFE(raviart_thomas,Float64,order) #THIS WORKS
#reffeₑ = ReferenceFE(nedelec,Float64,order) #DOES NOT WORK
V = FESpace(model, reffeₑ,vector_type=Vector{ComplexF64}
    , conformity=:H1
    #, conformity=:Hdiv
    #, conformity=:HCurl
    , dirichlet_tags=dirichlet_tags)

U = TrialFESpace(V,dirichlet_values)

TrialFESpace()

In [27]:
a(u,v) = ∫( (∇×v)⋅(∇×u) )*dΩ

a (generic function with 1 method)

In [28]:
m(u,v) = ∫(u⋅v)dΩ

m (generic function with 1 method)

In [29]:
include("eigen.jl")

solve (generic function with 1 method)

In [30]:
nev = 10 # number of eigenvalues asked to evaluate.
prob = EigenProblem(a, m, U, V; nev=nev )
#prob = EigenProblem(a, m, U, V; nev=nev, which=:LM, explicittransform=:auto, tol=10^(-6), maxiter=100, sigma=2.)
ξ, uₕs = solve(prob);

In [31]:
ξ

10-element Vector{ComplexF64}:
 -8.748570722335883e-17 + 5.07330486630189e-30im
  1.831384655974178e-16 - 1.0657961466159127e-29im
  2.345904004973977e-16 + 1.2413129309407462e-28im
  2.942650559294757e-16 + 2.40064160611875e-29im
 -3.419965452166116e-16 + 1.712150274579942e-28im
 4.0364174567160937e-16 + 2.3672455120759076e-29im
  5.715885360969915e-16 + 6.730570071505338e-29im
 -6.320600727065068e-16 - 7.949938600609733e-28im
 -6.819112247973592e-16 - 3.026682610536548e-28im
  8.711906365820084e-16 + 5.943114926637237e-30im

In [32]:
plot(sort(real(ξ)))

In [35]:
uₕ = uₕs[3]

SingleFieldFEFunction():
 num_cells: 400
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 181019260283380811

In [36]:
fig, ax, plt = plot(Ω, ( u-> real(u[1])) ∘ uₕ)
#fig, ax, plt = plot(Ω, real.(uₕ))
ax.aspect = AxisAspect(1)
Colorbar(fig[2,1], plt, vertical=false)
fig

┌ Error: error handling request
│   exception = (ErrorException("Metadata array needs to have same length as data.\n                    Found 2400 data items, and 1600 metadata items"), Base.StackTraces.StackFrame[error(s::String) at error.jl:35, meta(elements::Buffer{GeometryBasics.Point{2, Float32}, Vector{GeometryBasics.Point{2, Float32}}}; meta::Base.Pairs{Symbol, Buffer{Float32, Vector{Float32}}, Tuple{Symbol}, NamedTuple{(:color,), Tuple{Buffer{Float32, Vector{Float32}}}}}) at metadata.jl:125, (::GeometryBasics.var"#meta##kw")(::NamedTuple{(:color,), Tuple{Buffer{Float32, Vector{Float32}}}}, ::typeof(GeometryBasics.meta), elements::Buffer{GeometryBasics.Point{2, Float32}, Vector{GeometryBasics.Point{2, Float32}}}) at metadata.jl:118, create_shader(scene::Scene, plot::Mesh{Tuple{GeometryBasics.Mesh{2, Float32, GeometryBasics.Ngon{2, Float32, 3, GeometryBasics.Point{2, Float32}}, GeometryBasics.SimpleFaceView{2, Float32, 3, GeometryBasics.OffsetInteger{-1, UInt32}, GeometryBasics.P

In [13]:
l = 5
uₕ = uₕs[l]
writevtk(Ω,"eigenvector_$l",order=1,cellfields=["e_1"=>( u-> real(u[1])) ∘ uₕ, "e_2"=>( u-> real(u[2])) ∘ uₕ])

┌ Error: error handling request
│   exception = (ErrorException("Metadata array needs to have same length as data.\n                    Found 2400 data items, and 1600 metadata items"), Base.StackTraces.StackFrame[error(s::String) at error.jl:35, meta(elements::Buffer{GeometryBasics.Point{2, Float32}, Vector{GeometryBasics.Point{2, Float32}}}; meta::Base.Pairs{Symbol, Buffer{Float32, Vector{Float32}}, Tuple{Symbol}, NamedTuple{(:color,), Tuple{Buffer{Float32, Vector{Float32}}}}}) at metadata.jl:125, (::GeometryBasics.var"#meta##kw")(::NamedTuple{(:color,), Tuple{Buffer{Float32, Vector{Float32}}}}, ::typeof(GeometryBasics.meta), elements::Buffer{GeometryBasics.Point{2, Float32}, Vector{GeometryBasics.Point{2, Float32}}}) at metadata.jl:118, create_shader(scene::Scene, plot::Mesh{Tuple{GeometryBasics.Mesh{2, Float32, GeometryBasics.Ngon{2, Float32, 3, GeometryBasics.Point{2, Float32}}, GeometryBasics.SimpleFaceView{2, Float32, 3, GeometryBasics.OffsetInteger{-1, UInt32}, GeometryBasics.P

(["eigenvector_5.vtu"],)

┌ Error: error handling request
│   exception = (ErrorException("Metadata array needs to have same length as data.\n                    Found 2400 data items, and 1600 metadata items"), Base.StackTraces.StackFrame[error(s::String) at error.jl:35, meta(elements::Buffer{GeometryBasics.Point{2, Float32}, Vector{GeometryBasics.Point{2, Float32}}}; meta::Base.Pairs{Symbol, Buffer{Float32, Vector{Float32}}, Tuple{Symbol}, NamedTuple{(:color,), Tuple{Buffer{Float32, Vector{Float32}}}}}) at metadata.jl:125, (::GeometryBasics.var"#meta##kw")(::NamedTuple{(:color,), Tuple{Buffer{Float32, Vector{Float32}}}}, ::typeof(GeometryBasics.meta), elements::Buffer{GeometryBasics.Point{2, Float32}, Vector{GeometryBasics.Point{2, Float32}}}) at metadata.jl:118, create_shader(scene::Scene, plot::Mesh{Tuple{GeometryBasics.Mesh{2, Float32, GeometryBasics.Ngon{2, Float32, 3, GeometryBasics.Point{2, Float32}}, GeometryBasics.SimpleFaceView{2, Float32, 3, GeometryBasics.OffsetInteger{-1, UInt32}, GeometryBasics.P